In [8]:
import datetime
import sys
import pymc3 as pm
import theano.tensor as tt
import pickle
import pandas as pd
import numpy as np
import scipy.special
from multiprocessing import cpu_count


sys.path.append("../covid19_inference")
sys.path.append("..")

import data
from causal_covid.data import load_cases, load_infectiability
from causal_covid.utils import get_cps, day_to_week_matrix
from covid19_inference import Cov19Model
from covid19_inference.model import (
    lambda_t_with_sigmoids,
    kernelized_spread_gender,
    week_modulation,
    student_t_likelihood,
    delay_cases,
    uncorrelated_prior_E,
)

begin = datetime.datetime(2021,8,1)
end = datetime.datetime(2021,10,1)
file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/observed_infection_data.csv"
cases_df = load_cases(file, begin, end, 1)


vaccination_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/observed_vaccination_data.csv"
waning_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/vaccine_efficacy_waning_data.csv"
population_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/population_data.csv"
observed_U2_file = "./../data/2022-02-02_14-19-24_observed_vac_policy/vaccination_policy/U_2.npy"
observed_U3_file = "./../data/2022-02-02_14-19-24_observed_vac_policy/vaccination_policy/u_3.npy"
scenario_U2_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/vaccination_policy/U_2.npy"
scenario_U3_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/vaccination_policy/u_3.npy"

#pd.read_csv(waning_file)

infectiability = load_infectiability(vaccination_file, population_file, observed_U2_file, observed_U3_file, waning_file, begin, end, 1)

In [9]:
cases_df

,0+
2021-08-01,23496.0
2021-08-08,38015.0
2021-08-15,51069.0
2021-08-22,60935.0
2021-08-29,65742.0
2021-09-05,54958.0
2021-09-12,54295.0
2021-09-19,40245.0
2021-09-26,24884.0


In [10]:
infectiability

,0+
2021-08-01,0.486162
2021-08-08,0.470329
2021-08-15,0.444959
2021-08-22,0.417638
2021-08-29,0.385708
2021-09-05,0.398584
2021-09-12,0.392542
2021-09-19,0.392823
2021-09-26,0.376308
